In [1]:
# Environnement VAR
GCP_PROJECT='the-mdr-project'
GCP_DB='live_mdr'
# CAT
CAT_TABLE='category_dataset'
CAT_TABLE_ID=f'{GCP_PROJECT}.{GCP_DB}.{CAT_TABLE}'
# NEWS
NEWS_TABLE='news_dataset'
NEWS_TABLE_ID=f'{GCP_PROJECT}.{GCP_DB}.{NEWS_TABLE}'
# USER
USER_TABLE='user_dataset'
USER_TABLE_ID=f'{GCP_PROJECT}.{GCP_DB}.{USER_TABLE}'

In [2]:
from google.cloud import bigquery
from google.oauth2 import service_account

# Path to your service account key file
key_path = "../credentials/mathieu_credentials.json"

# Create credentials using the key file
credentials = service_account.Credentials.from_service_account_file(key_path)

client = bigquery.Client(credentials=credentials, project=GCP_PROJECT)

# RETRIEVE ONE RANDOM NEWS

In [28]:
USER_ID = 1 #mathieu
categories = [1, 3, 5]
params = [
    bigquery.ArrayQueryParameter("category_ids", "INT64", categories),
    bigquery.ScalarQueryParameter("user_id", "INT64", USER_ID),
    bigquery.ScalarQueryParameter("nb_news", "INT64", 20),
]

In [29]:
query = """
    WITH CombinedNews AS (
        SELECT *,
               ROW_NUMBER() OVER(ORDER BY RAND()) AS rand_num
        FROM `the-mdr-project.live_mdr.news_dataset`
        WHERE category_id IN UNNEST(@category_ids)
            AND news_id NOT IN (
                SELECT news_id
                FROM `the-mdr-project.live_mdr.review_dataset`
                WHERE user_id = @user_id
            )
    )
    SELECT *
    FROM CombinedNews
    WHERE rand_num <= @nb_news;
    """

job_config = bigquery.QueryJobConfig()
job_config.query_parameters = params

query_job = client.query(query, job_config=job_config)

results = query_job.result()
df = results.to_dataframe()
df.to_dict()

{'news_id': {0: 151165,
  1: 226516,
  2: 306071,
  3: 312657,
  4: 117169,
  5: 256822,
  6: 223090,
  7: 258730,
  8: 243662,
  9: 238542,
  10: 194239,
  11: 250488,
  12: 239855,
  13: 116865,
  14: 255576,
  15: 235516,
  16: 88719,
  17: 50178,
  18: 168211,
  19: 319192},
 'category_id': {0: 5,
  1: 1,
  2: 5,
  3: 5,
  4: 5,
  5: 1,
  6: 1,
  7: 3,
  8: 1,
  9: 1,
  10: 1,
  11: 1,
  12: 1,
  13: 5,
  14: 1,
  15: 1,
  16: 1,
  17: 1,
  18: 5,
  19: 5},
 'title': {0: 'Dinan Sport Cycling : « On a de quoi bien jouer »',
  1: 'Game of Rôles x OTP - Sentinelles - Episode 4 (partie 1)',
  2: '#88 "Cap sur Harricana", l\'UTHC d\'Alix Noblat et Yvan L\'Heureux',
  3: 'Les Bleus assurent',
  4: 'Chris Boucher souffre d’une déchirure partielle au genou droit',
  5: None,
  6: "Jean-Marie Le Pen, l'obsession nationale 5/7 : Le vol noir du corbeau (1987-1998)",
  7: 'Ciné court animé de Roanne : l’animation à l’honneur en mars',
  8: 'Marvel : on connaît les salaires hallucinants de Robe